In [9]:
import numpy as np 
import tensorflow as tf
import os
import utils
import pickle

!wandb login "41c25b4fc8e96d4ae0d96e0abd4d69787a6ea35f"
import wandb
from wandb.keras import WandbCallback
wandb.init(project="text-generator-esc190")
config = wandb.config

Successfully logged in to Weights & Biases!
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\DS/.netrc


In [10]:
#importing word dictionaries 

vec2word = pickle.load(open("normedvec2word512.p", "rb"))
word2vec = pickle.load(open("normedword2vec512.p", "rb"))
VECTOR_SIZE = 512

In [6]:
'''
vec2word = {tuple(key / np.sum(key)): value.lower() for key, value in vec2word.items()}
word2vec = {key.lower(): value[0] / np.sum(value[0]) for key, value in word2vec.items()}
'''

In [7]:
'''
#generating random embeddings for punctuation
punc2embed = {}
for each in utils.punc_list:
    notnormed = np.random.randn(VECTOR_SIZE)
    punc2embed[each] = notnormed / np.sum(notnormed)
    vec2word[tuple(punc2embed[each])] = each
    word2vec[each] = punc2embed[each]
pickle.dump(punc2embed, open("randpunc2embed16.p", "wb"))
'''

In [8]:
'''
pickle.dump(vec2word, open("normedvec2word16.p", "wb"))
pickle.dump(word2vec, open("normedword2vec16.p", "wb"))
'''

In [6]:
'''
keys = list(word2vec.keys())
values = [word2vec[each] for each in keys]

keys = tf.constant(keys, dtype=tf.string)
values = tf.constant(values)
print(values[0].shape)

tf_word2vec = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=keys,
        values=values,
    ),
    default_value=tf.zeros(()),
    name="num2wordlookup"
)
'''

'\nkeys = list(word2vec.keys())\nvalues = [word2vec[each] for each in keys]\n\nkeys = tf.constant(keys, dtype=tf.string)\nvalues = tf.constant(values)\nprint(values[0].shape)\n\ntf_word2vec = tf.lookup.StaticHashTable(\n    initializer=tf.lookup.KeyValueTensorInitializer(\n        keys=keys,\n        values=values,\n    ),\n    default_value=tf.zeros(()),\n    name="num2wordlookup"\n)\n'

In [ ]:
'''
characterSet = []
for each in list(word2vec.keys()):
    characterSet.extend(list(each))
characterSet = set(characterSet)
characterSet = list(characterSet)
characterSet.sort()
print(characterSet)
pickle.dump(characterSet, open("charlist.p", "wb"))
'''

In [8]:
'''
#generating random embeddings for punctuation
punc2embed = {}
for each in utils.punc_list:
    notnormed = np.random.randn(VECTOR_SIZE)
    punc2embed[each] = notnormed / np.sum(notnormed)
pickle.dump(punc2embed, open("randpunc2embed.p", "wb"))
'''

'\n#generating random embeddings for punctuation\npunc2embed = {}\nfor each in utils.punc_list:\n    notnormed = np.random.randn(VECTOR_SIZE)\n    punc2embed[each] = notnormed / np.sum(notnormed)\npickle.dump(punc2embed, open("randpunc2embed.p", "wb"))\n'

In [11]:
def processText(inputfile, outputfile):
    #text processing
    charlist = pickle.load(open("charlist.p", "rb"))
    textfile = inputfile
    outputfn = outputfile
    fileyboi = open(textfile, "r", encoding="utf8")
    outputfile = open(outputfn, "w")

    outputlist = []

    for char in fileyboi.read():
        char = char.lower()
        if char in utils.punc_list or char in charlist or char == " ":
            outputlist.append(char)
    outputstr = "".join(outputlist)
    for each in utils.punc_list:
        outputstr = outputstr.replace(each, " {} ".format(each))

    outputstr = outputstr.split(" ")
    outputstr = " ".join(outputstr)

    outputlist = []

    for each in outputstr.split(" "):
        if each in word2vec:
            outputlist.append(each)
    outputstr = " ".join(outputlist)
    outputfile.write(outputstr)
    fileyboi.close()
    outputfile.close()


In [10]:
#processText("nosleep.txt", "processednosleep.txt")

In [11]:
#loading text in
#creating function to be mapped
#uses word2vec
def str2embeddedvec_py(string):
    output = word2vec[string.numpy().decode("ascii")]
    return output

def str2embeddedvec(string):
    [output,] = tf.py_function(str2embeddedvec_py, inp=[string], Tout=[tf.float32])
    output.set_shape(VECTOR_SIZE)
    return output

def split2chunks(inp):
    input_vec = inp[:-1]
    target_vec = inp[1:]
    return input_vec, target_vec

sequence_length = 16
BUFFER_SIZE = 5000
BATCH_SIZE = 32

In [13]:
#opening file
filename = "processednosleep.txt"
with open(filename, "r") as processedFile: 
    dataset = tf.data.Dataset.from_tensor_slices(np.array(processedFile.read().split(" ")))
    dataset = dataset.map(str2embeddedvec)
    dataset = dataset.batch(sequence_length+1, drop_remainder=True)
    dataset = dataset.map(split2chunks)
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [13]:
#wowies = dataset.take(1)
#print(np.array(list(wowies.as_numpy_iterator())).shape)

In [14]:
#creating model
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import cosine_similarity

NUM_UNITS = 2000
inputs = Input(batch_shape=(BATCH_SIZE, sequence_length, VECTOR_SIZE))
LSTM1 = LSTM(NUM_UNITS, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')(inputs)
output = TimeDistributed(Dense(VECTOR_SIZE))(LSTM1)

model = Model(inputs=inputs, outputs=output)
optimizer = Adam(learning_rate = 0.0003)

model.compile(loss = cosine_similarity, optimizer = optimizer, metrics=["cosine_similarity"])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(32, 16, 32)]            0         
_________________________________________________________________
lstm (LSTM)                  (32, 16, 2000)            16264000  
_________________________________________________________________
time_distributed (TimeDistri (32, 16, 32)              64032     
Total params: 16,328,032
Trainable params: 16,328,032
Non-trainable params: 0
_________________________________________________________________


In [15]:
config.batch_size = BATCH_SIZE
config.epochs = 20

In [16]:
#model.load_weights("512nosleep.1.09.h5")

In [16]:
from tensorflow.keras.callbacks import ModelCheckpoint
save_model = ModelCheckpoint(filepath="32nosleep.1.{epoch:02d}.h5", monitor='cosine_similarity', save_weights_only=True, mode='auto', verbose=1)

In [17]:
wandb.init()
model.fit(dataset, epochs=config.epochs, callbacks=[WandbCallback(), save_model])

Train for 2895 steps
Epoch 1/20
2894/2895 [============================>.] - ETA: 0s - loss: -0.3030 - cosine_similarity: 0.3030
Epoch 00001: saving model to 32nosleep.1.01.h5
2895/2895 [==============================] - 323s 111ms/step - loss: -0.3030 - cosine_similarity: 0.3030
Epoch 2/20
2894/2895 [============================>.] - ETA: 0s - loss: -0.3231 - cosine_similarity: 0.3231
Epoch 00002: saving model to 32nosleep.1.02.h5
2895/2895 [==============================] - 316s 109ms/step - loss: -0.3231 - cosine_similarity: 0.3231
Epoch 3/20
2894/2895 [============================>.] - ETA: 0s - loss: -0.3281 - cosine_similarity: 0.3281
Epoch 00003: saving model to 32nosleep.1.03.h5
2895/2895 [==============================] - 321s 111ms/step - loss: -0.3281 - cosine_similarity: 0.3281
Epoch 4/20
2894/2895 [============================>.] - ETA: 0s - loss: -0.3308 - cosine_similarity: 0.3308
Epoch 00004: saving model to 32nosleep.1.04.h5
2895/2895 [==============================] -

In [30]:
from numpy import linalg as LA

def vec2predictions(inputvec, word_arr, vec_arr, temperature = 0.005, prob = True, num_samples = 1):
    '''
    inputvec => np.array of shape (variable, VECTOR_SIZE)
    word_arr => np.array of shape (vocab_size, 1)
    vec_arr => np.array of shape (vocab_size, VECTOR_SIZE)
    returns a np.array of shape (variable), with words
    '''
    dotted_prod = inputvec @ vec_arr.T
    norm_vec_arr = LA.norm(vec_arr, axis=1)
    norm_inputvec = LA.norm(inputvec, axis=1)
    divisor = np.expand_dims(norm_inputvec.T, 1) @ np.expand_dims(norm_vec_arr, 0)
    '''
    print("computed divisor with shape", divisor.shape)
    print("computed dotted_prod with shape", dotted_prod.shape)
    '''
    finalvec = dotted_prod / divisor
    finalvec = finalvec / temperature

    if prob:
        indicies = tf.random.categorical(finalvec, num_samples=num_samples)
    else:
        indicies = np.argsort(-finalvec)[:num_samples]
    indicies = tf.squeeze(indicies).numpy()

    #print("computed final prediction vector: with shape", indicies.shape)

    output = word_arr[indicies]
    return output

def processString(inputstr, punc_list, charlist, word2vec):
    '''
    returns a list of fully processed strings with guarenteed mappings with alpha num chars and punctuation
    '''
    outputlist = []
    for char in inputstr:
        char = char.lower()
        if char in punc_list or char in charlist or char == " ":
            outputlist.append(char)

    outputstr = "".join(outputlist)

    for each in punc_list:
        outputstr = outputstr.replace(each, " {} ".format(each))

    outputstr = outputstr.split(" ")
    outputstr = " ".join(outputstr)

    outputlist = []

    for each in outputstr.split(" "):
        if each in word2vec:
            outputlist.append(each)
    
    return outputlist


def generateText(model, start_string, word2vec, punc_list, char_list, num_generate = 15, temperature = 0.005):
    word_arr = []
    vec_arr = []
    for word, vector in word2vec.items():
        word_arr.append(word)
        vec_arr.append(vector)
    word_arr = np.array(word_arr)
    vec_arr = np.array(vec_arr)

    inputvec = []
    for each in processString(start_string, punc_list, char_list, word2vec):
        inputvec.append(word2vec[each])

    inputvec = np.array(inputvec)

    inputvec = tf.expand_dims(inputvec, 0)

    outputText = []
    model.reset_states()
    for i in range(num_generate):
        #print("starting prediction")
        pred = model.predict(inputvec)
        #print("finished prediction: output shape = ", pred.shape)
        # divide prediction by temperature to do stuff
        pred = tf.expand_dims(pred[:, -1, :], 0)
        pred = tf.squeeze(pred, 0)
        predicted_word = vec2predictions(pred, word_arr, vec_arr, temperature)
        #print("predicted word is: ", predicted_word)
        outputText.append(predicted_word)

        inputvec = tf.expand_dims(tf.expand_dims(word2vec[predicted_word], 0), 0)

    return (start_string + " ".join(outputText))

In [12]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input, LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import cosine_similarity

NUM_UNITS = 2000
inputs = Input(batch_shape=(1, None, VECTOR_SIZE))
LSTM1 = LSTM(NUM_UNITS, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')(inputs)
output = TimeDistributed(Dense(VECTOR_SIZE))(LSTM1)

model = Model(inputs=inputs, outputs=output)
optimizer = Adam(learning_rate = 0.0003)

model.compile(loss = cosine_similarity, optimizer = optimizer, metrics=["cosine_similarity"])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(1, None, 512)]          0         
_________________________________________________________________
lstm (LSTM)                  (1, None, 2000)           20104000  
_________________________________________________________________
time_distributed (TimeDistri (None, None, 512)         1024512   
Total params: 21,128,512
Trainable params: 21,128,512
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.load_weights("512nosleep.2.10.h5")
#model.build(tf.TensorShape([1, None]))

In [14]:
model.save("512generatormodel.h5")

In [33]:
charlist = pickle.load(open("charlist.p", "rb"))
print(generateText(model, "remember i there are two odd things about i live", word2vec, utils.punc_list, charlist, num_generate=50))

remember i there are two odd things about i live. ganoin overlordship stun_gun stun_gun . 
 
 speakership egg_timer scrubber thaneship egg_timer stun_gun rope_yarn egg_timer . 
 
 entlebucher egg_timer riddle , movement stun_gun egg_timer . 
 
 speakership egg_timer trucking_company thaneship . ganoin supreme_allied_commander_europe supreme_allied_commander_europe . 
 
 aleph hypocritical riddle thaneship airline delta_ray . 
 
 speakership
